In [144]:
import os
import glob
from tqdm import tqdm

In [145]:
#DATA_PATH = os.getcwd() + r'\rawdata'
DATA_PATH = os.getcwd() + r'\target_data'
print(DATA_PATH)

C:\Users\Kislee\PycharmProjects\Korean_textbook\target_data


In [146]:
files = glob.glob(os.path.join(DATA_PATH, '*.xlsx'))

In [147]:
print(files)

['C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\경희대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\고려대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\서울대_5급_6급_학문목적듣기_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\성균관대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\연세대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\이화여대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\중앙대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\한국외대_5급_6급_ver1.0.xlsx', 'C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\한양대_5급_6급_ver1.0.xlsx']


In [148]:
import pandas as pd

In [149]:
all_dfs = []
for source in files:
    temp_df = pd.read_excel(source, engine='openpyxl')
    all_dfs.append(temp_df)
df = pd.concat(all_dfs)

In [150]:
df.head()

,학교명,등급,지문
0,경희대,5,장민: 어제 요리 프로그램에서 제철 음식을 가지고 누가 더 맛 있게 만드는지 대결을...
1,경희대,5,"흐엉: 장민, 이 제육볶음 좀 먹어 봐. 어제 마을 장터에 갔는데 직접 만든 고추장..."
2,경희대,5,은영: 승진 시험 준비는 잘 되어 가?\n상진: 그냥 그래. 요즘 일도 공부도 손에...
3,경희대,5,의학 전문가: 여러분은 어떻게 휴식을 하십니까? 집에서 텔레비전을 보면서 쉬신다고요...
4,경희대,5,예술 교육은 우리의 삶에 어떤 영향을 줄까요? 그에 대한 하나의 대답으로 콜롬비아의...


In [151]:
len(df)

288

In [152]:
#-*- coding:utf-8 -*-
import urllib3
import json

In [153]:
# 언어 분석 기술 문어/구어 중 한가지만 선택해 사용
# 언어 분석 기술(문어)
#openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 
# 언어 분석 기술(구어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
 
accessKey = "37cdd2da-8145-47be-ac46-caae6b04fdcc"
analysisCode = "ner"

In [154]:
def requestNERfromText(text):
    #요청정보 생성
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }
        
    http = urllib3.PoolManager()
    #API 호출
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
        
    #수신 데이터 변환
    if response.status ==  200:
        result = json.loads(response.data)
    else:
        result = None        
    
    #개체정보 수집
    ner_list = []
    if result != None:
        for sent in result['return_object']['sentence']:
            for item in sent['NE']:
                ner_list.append((item['text'], item['type']))
    
    return ner_list   

In [155]:
def isName(tag):
    if tag == 'PS_NAME':
        return True
    else:
        return False

def isContry(tag):
    if tag == 'LCP_COUNTRY':
        return True
    else:
        return False

def isTribe(tag):
    if tag == 'CV_TRIBE':
        return True
    else:
        return False

In [156]:
#추가할 컬럼들
name_columns = [] #외국인인명
contry_columns = [] #국가명
tribe_columns = [] #민족명

In [157]:
attribute='지문'
for cell in tqdm(df[attribute]):
    ner_info = []
    try:
        ner_info = requestNERfromText(cell)
    except:
        print('NER 정보 수집 실패')
    
    names_in_cell = []
    contry_in_cell = []
    tribe_in_cell = []
    for word, tag in ner_info:
        if isName(tag):
            names_in_cell.append(word)
        elif isContry(tag):
            contry_in_cell.append(word)
        elif isTribe(tag):
            tribe_in_cell.append(word)

    name_columns.append(list(set(names_in_cell)))
    contry_columns.append(list(set(contry_in_cell)))
    tribe_columns.append(list(set(tribe_in_cell)))

        

100%|████████████████████████████████████████████████████████████████████████████████| 288/288 [02:29<00:00,  1.93it/s]


In [158]:
name_columns = ['/'.join(item) for item in name_columns]
contry_columns = ['/'.join(item) for item in contry_columns]
tribe_columns = ['/'.join(item) for item in tribe_columns]

In [159]:
df['외국인인명'] = name_columns
df['국가명'] = contry_columns
df['민족명'] = tribe_columns

In [160]:
df.to_excel('result.xlsx')